# Peformance comparison of different sparse data formats 

In this notebook we analyze the performance of various algorithms, including MLPs, dense dynamical systems, and various sorts of sparse dynamical systems.

In [14]:
import torch
import warnings
import time
warnings.filterwarnings("ignore")
# TODO Add these imports and test MaskeLinear and SparseLinear
from iterativennsimple.MaskedLinear import MaskedLinear
from iterativennsimple.SparseLinear import SparseLinear

import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
results = {}

In [3]:
def generate(size, entries, device="cuda"):
    """create a variety of sparse matrices 

    Args:
        size (int, optional): Size of the square matrix. Defaults to 1000.
        entries (int, optional): Total number of non-zero entries. Note this is an upper bound, but should be close to the actual size. Defaults to 23*1000.
        device (str, optional): "cuda" or "cpu". Defaults to "cuda".

    Returns:
        dict: Dictionary of the sparse matrices
    """ 
    output = {}
    # We first create a COO tensor since that is easier to create
    indices = torch.randint(0, size, (entries,2))
    vals = torch.randn(entries)
    coo = torch.sparse_coo_tensor(indices.t(), vals, (size, size), device=device)
    coo = coo.coalesce()

    # Then we convert it to CSC, CSR and dense
    dense = coo.to_dense()
    csc = coo.to_sparse_csc()
    csr = coo.to_sparse_csr()
    
    # We also create the MaskedLinear and SparseLinear objects
    class moduleWrapper(object):
        def __init__(self, module, device=device):
            self.module = module
            self.device = device
        def __matmul__(self, x):
            return self.module(x.T).T
        
    maskedLinear = moduleWrapper(MaskedLinear.from_coo(coo).to(device), device)
    sparseLinear = moduleWrapper(SparseLinear.from_coo(coo).to(device), device)

    output["dense"] = dense
    output["coo"] = coo
    output["csc"] = csc
    output["csr"] = csr
    output["maskedLinear"] = maskedLinear
    output["sparseLinear"] = sparseLinear
    return output

In [4]:
## check that all of the matrices are the same operator
def matrix_check(size, entries, device):
    # Generate the matrices
    matrices = generate(size, entries, device=device)
    # Size of the RHS
    x_cols = 100
    x = torch.randn(size, x_cols, device=device)

    print('Matrix-matrix multiplication check')
    y_true = None
    base_name = None

    for matrix, matrix_name in zip(matrices.values(), matrices.keys()):
        y = matrix @ x
        if y_true is None:
            y_true = y
            base_name = matrix_name
        else:
            # print the frobenius norm of the difference
            print(f"||{matrix_name} - {base_name}||_F: {torch.norm(y-y_true)}")
            

In [5]:
matrix_check(1000, 23*1000, "cpu")

Matrix-matrix multiplication check
||coo - dense||_F: 0.0001550955348648131
||csc - dense||_F: 0.00016146268171723932
||csr - dense||_F: 0.00016146268171723932
||maskedLinear - dense||_F: 3.2562944397795945e-05
||sparseLinear - dense||_F: 0.00016146268171723932


In [6]:
matrix_check(1000, 23*1000, "cuda")

Matrix-matrix multiplication check
||coo - dense||_F: 0.00014411909796763211
||csc - dense||_F: 0.00014411758456844836
||csr - dense||_F: 0.0001441066706320271
||maskedLinear - dense||_F: 0.0001318515423918143
||sparseLinear - dense||_F: 0.00016621372196823359


In [7]:
def run_timing(size, entries, device, syncgpu):
    # test if cuda is available
    if device == "cuda":
        if not torch.cuda.is_available():
            print("CUDA is not available, using CPU instead")
            device = "cpu"

    print(f"Running on {device}")
    print(f"Size: {size}")
    print(f"Entries: {entries}")
    print(f"Synchronize GPU: {syncgpu}")
    
    # Generate the matrices
    matrices = generate(size, entries, device=device)

    # Size of the RHS
    x_cols = 100

    # Compute the timings
    print('Matrix-matrix multiplication timings:')
    base_time = None
    base_name = None
    all_times = {}
    for matrix, matrix_name in zip(matrices.values(), matrices.keys()):
        # first, do a few runs to warm up the cache
        for i in range(2):
            x = torch.randn(size, x_cols, device=device)
            y = matrix @ x
        # now do the timings
        matrix_times = []
        for i in range(5):
            x = torch.randn(size, x_cols, device=device)    
            if syncgpu:
                torch.cuda.synchronize()
            start = time.perf_counter()
            y = matrix @ x
            if syncgpu:
                torch.cuda.synchronize()
            matrix_time = time.perf_counter()-start
            matrix_times.append(matrix_time)
        avg_matrix_time = sum(matrix_times)/len(matrix_times)
        min_matrix_time = min(matrix_times)
        max_matrix_time = max(matrix_times)
        if base_time is None:
            base_time = avg_matrix_time
            base_name = matrix_name

        name = device+'_'+matrix_name
        all_times[name] = avg_matrix_time

        print(f"{name} avg time:", avg_matrix_time)
        print(f"{name} min time:", min_matrix_time)
        print(f"{name} max time:", max_matrix_time)
        print(f"{name} speedup over {base_name}:", base_time/avg_matrix_time)


    results.update(all_times)
    return all_times

In [8]:
matrix_size = 500
matrix_entries = 500*matrix_size
# matrix_size = 20000
# matrix_entries = 100*matrix_size
print("matrix size:", matrix_size)
print("Total number of entries:", matrix_size**2)
print("Active entries:", matrix_entries)
print("Ratio of active entries to total entries:", matrix_entries/(matrix_size**2))


matrix size: 500
Total number of entries: 250000
Active entries: 250000
Ratio of active entries to total entries: 1.0


In [9]:
all_times = run_timing(matrix_size, matrix_entries, "cpu", True)

Running on cpu
Size: 500
Entries: 250000
Synchronize GPU: True


Matrix-matrix multiplication timings:
cpu_dense avg time: 0.00010764836333692074
cpu_dense min time: 0.00010507798288017511
cpu_dense max time: 0.0001143548870459199
cpu_dense speedup over dense: 1.0
cpu_coo avg time: 0.00488598442170769
cpu_coo min time: 0.00484867300838232
cpu_coo max time: 0.004929715069010854
cpu_coo speedup over dense: 0.02203207256630933
cpu_csc avg time: 0.0028228267794474958
cpu_csc min time: 0.002752267988398671
cpu_csc max time: 0.0029443189268931746
cpu_csc speedup over dense: 0.038134951857722726
cpu_csr avg time: 0.00020731783006340266
cpu_csr min time: 0.00019508705008774996
cpu_csr max time: 0.00021621608175337315
cpu_csr speedup over dense: 0.5192431509822351
cpu_maskedLinear avg time: 0.00022037478629499674
cpu_maskedLinear min time: 0.00021616602316498756
cpu_maskedLinear max time: 0.00022660696413367987
cpu_maskedLinear speedup over dense: 0.4884785829937058
cpu_sparseLinear avg time: 0.013665704196318984
cpu_sparseLinear min time: 0.0129041170002892

In [10]:
all_times = run_timing(matrix_size, matrix_entries, "cuda", True)

Running on cuda
Size: 500
Entries: 250000
Synchronize GPU: True
Matrix-matrix multiplication timings:
cuda_dense avg time: 2.335601020604372e-05
cuda_dense min time: 2.184102777391672e-05
cuda_dense max time: 2.6089022867381573e-05
cuda_dense speedup over dense: 1.0
cuda_coo avg time: 7.808499503880739e-05
cuda_coo min time: 7.744599133729935e-05
cuda_coo max time: 7.972004823386669e-05
cuda_coo speedup over dense: 0.2991100939999553
cuda_csc avg time: 0.0003505416214466095
cuda_csc min time: 0.00034844502806663513
cuda_csc max time: 0.00035221304278820753
cuda_csc speedup over dense: 0.06662835103477446
cuda_csr avg time: 3.975860308855772e-05
cuda_csr min time: 3.7520076148211956e-05
cuda_csr max time: 4.718801937997341e-05
cuda_csr speedup over dense: 0.5874454430408657
cuda_maskedLinear avg time: 7.570257876068353e-05
cuda_maskedLinear min time: 7.266702596098185e-05
cuda_maskedLinear max time: 8.298596367239952e-05
cuda_maskedLinear speedup over dense: 0.3085233104129574
cuda_spar

In [18]:
# Create a dataframe with row and column names for the heatmap
df = pd.DataFrame(columns=results.keys(), index=results.keys())

# Fill in df with the values we want to display
for name1, time1 in results.items():
    for name2, time2 in results.items():
        df.loc[name1, name2] = time1/time2
# Create the heatmap and make it large
# Plot the original heatmap
fig = px.imshow(df, text_auto=True, color_continuous_scale='Jet', width=1000, height=900, title="Relative Timing Heatmap")
fig.show()

# Plot the log10 heatmap
df_log = np.log10(df.astype(float))
fig_log = px.imshow(df_log, text_auto=True, color_continuous_scale='Jet', width=1000, height=900, title="Log10 Relative Timing Heatmap")
fig_log.show()

In [12]:
def train_least_squares(size, entries, device, syncgpu, epochs=30):
    # test if cuda is available
    if device == "cuda":
        if not torch.cuda.is_available():
            print("CUDA is not available, using CPU instead")
            device = "cpu"

    print(f"Running on {device}")
    print(f"Size: {size}")
    print(f"Entries: {entries}")
    print(f"Synchronize GPU: {syncgpu}")
    
    # Generate the matrices
    matrices = generate(size, entries, device=device)

    # Generate the data
    X = torch.randn(10, size, device=device)  # Example input data
    Y = torch.randn(10, size, device=device)  # Example target data

    # Define the loss function
    criterion = torch.nn.MSELoss()

    # Wrap a model around the sparse matrix
    class matrixWrapper(torch.nn.Module):
        def __init__(self, matrix, type="dense"):
            super(matrixWrapper, self).__init__()
            self.type = type
            self.matrix = torch.nn.Parameter(matrix)
        def forward(self, x):
            if self.type == "dense":
                return torch.mm(x, self.matrix.T)
            else:
                return torch.sparse.mm(self.matrix, x.T).T

    # Train the models
    print('Training least squares models:')
    for matrix, matrix_name in zip(matrices.values(), matrices.keys()):
        if matrix_name in ["coo", "csr"]:
            sparse_model = matrixWrapper(matrix, type="sparse").to(device)
        elif matrix_name in ["dense"]:
            sparse_model = matrixWrapper(matrix, type="dense").to(device)
        elif matrix_name in ["maskedLinear", "sparseLinear"]:
            sparse_model = matrix.module.to(device)
        else:
            continue

        # Define the optimizer
        optimizer = torch.optim.SGD(sparse_model.parameters(), lr=0.01)

        # Training loop
        print(f'-------------{matrix_name} training:--------------')
        for epoch in range(epochs):
            optimizer.zero_grad()
            output = sparse_model(X)
            loss = criterion(output, Y)
            if epoch % 10 == 0:
                print(f'{matrix_name} Epoch {epoch}, Loss: {loss.item()}')
            loss.backward()
            optimizer.step()

        print(f'{matrix_name} final loss: {loss.item()}')
    return matrices

# Example usage
matrices = train_least_squares(size=100, entries=500, device='cuda', syncgpu=True)

Running on cuda
Size: 100
Entries: 500
Synchronize GPU: True
Training least squares models:
-------------dense training:--------------
dense Epoch 0, Loss: 6.207830905914307
dense Epoch 10, Loss: 5.944512844085693
dense Epoch 20, Loss: 5.693314552307129
dense final loss: 5.47710657119751
-------------coo training:--------------
coo Epoch 0, Loss: 6.207830905914307
coo Epoch 10, Loss: 6.168856620788574
coo Epoch 20, Loss: 6.130222797393799
coo final loss: 6.095741271972656
-------------csr training:--------------
csr Epoch 0, Loss: 6.207830905914307
csr Epoch 10, Loss: 6.168856620788574
csr Epoch 20, Loss: 6.130222797393799
csr final loss: 6.095741271972656
-------------maskedLinear training:--------------
maskedLinear Epoch 0, Loss: 6.207830905914307
maskedLinear Epoch 10, Loss: 6.168856620788574
maskedLinear Epoch 20, Loss: 6.130222797393799
maskedLinear final loss: 6.095741271972656
-------------sparseLinear training:--------------
sparseLinear Epoch 0, Loss: 6.207830905914307
sparse